# Web Scraping

In [17]:
import pandas as pd
import seaborn as sns
from requests import get
from bs4 import BeautifulSoup

url = 'http://www.imdb.com/search/title?release_date=2017&sort=num_votes,desc&page=1'
response = get(url)
print(response.text[:500])





<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///?src=mdot">



        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>

<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle"


In [18]:

page = get("http://dataquestio.github.io/web-scraping-pages/simple.html")
print(page)
print(page.status_code)
print(page.content)

<Response [200]>
200
b'<!DOCTYPE html>\n<html>\n    <head><style>ytd-live-chat-frame#chat { display: none; }</style><style>#watch-discussion, #watch7-discussion, ytd-comments { display: none; }</style>\n        <title>A simple example page</title>\n    </head>\n    <body>\n        <p>Here is some simple content for this page.</p>\n    </body>\n</html>'


In [19]:
soup = BeautifulSoup(page.content, 'html.parser')
type(soup)

bs4.BeautifulSoup

In [20]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <style>
   ytd-live-chat-frame#chat { display: none; }
  </style>
  <style>
   #watch-discussion, #watch7-discussion, ytd-comments { display: none; }
  </style>
  <title>
   A simple example page
  </title>
 </head>
 <body>
  <p>
   Here is some simple content for this page.
  </p>
 </body>
</html>


In [21]:
list(soup.children)

print([type(item) for item in list(soup.children)])
 
html = list(soup.children)[1]
print(html)

[<class 'bs4.element.Doctype'>, <class 'bs4.element.NavigableString'>, <class 'bs4.element.Tag'>]




In [22]:
body = list(soup.children)[2]
print(body)
list(body.children)
print(list(body.children)[1])

<html>
<head><style>ytd-live-chat-frame#chat { display: none; }</style><style>#watch-discussion, #watch7-discussion, ytd-comments { display: none; }</style>
<title>A simple example page</title>
</head>
<body>
<p>Here is some simple content for this page.</p>
</body>
</html>
<head><style>ytd-live-chat-frame#chat { display: none; }</style><style>#watch-discussion, #watch7-discussion, ytd-comments { display: none; }</style>
<title>A simple example page</title>
</head>


In [23]:
p = list(body.children)[1]
print(p.get_text())


ytd-live-chat-frame#chat { display: none; }#watch-discussion, #watch7-discussion, ytd-comments { display: none; }
A simple example page



# Finding all instances of a tag

In [24]:
soup = BeautifulSoup(page.content, 'html.parser')
soup.find_all('p')

[<p>Here is some simple content for this page.</p>]

In [25]:
soup.find('p')

<p>Here is some simple content for this page.</p>

# Searching for tags by class and id